In [58]:
import os
import json
from tqdm.auto import tqdm
from dotenv import load_dotenv

import minsearch
from openai import OpenAI

import tiktoken

load_dotenv() 

True

In [26]:
with open("documents.json",'rt') as f:
    docs_raw = json.load(f)

In [27]:
documents = []

for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)

In [28]:
from elasticsearch import Elasticsearch

In [29]:
es_client = Elasticsearch("http://localhost:9200/")

In [30]:
es_client.info()

ObjectApiResponse({'name': 'd21a3ad48565', 'cluster_name': 'docker-cluster', 'cluster_uuid': 'aaOELs7ZTGeNslMjsC3p0Q', 'version': {'number': '8.4.3', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '42f05b9372a9a4a470db3b52817899b99a76ee73', 'build_date': '2022-10-04T07:17:24.662462378Z', 'build_snapshot': False, 'lucene_version': '9.3.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

In [31]:
query = 'How do I execute a command in a running docker container?'

In [32]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = "course-homework"

es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-homework'})

In [41]:
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 948/948 [00:01<00:00, 534.33it/s]


In [47]:
response = es_client.search(
    index=index_name,
    body={
        'query': {
            'bool': {
                'must': {
                    'multi_match': {
                        'query': query,
                        'fields': ['question^4', 'text'],
                        'type': 'best_fields'
                    }
                },
                'filter': [
                    {'term': {'course': 'machine-learning-zoomcamp'}}
                ]
            }
        }
    }
)

In [54]:
response.body['hits']['max_score']

84.050095

In [42]:
response = es_client.search(
    index=index_name,
    body={
        'query': {
            'bool': {
                'must': {
                    'multi_match': {
                        'query': query,
                        'fields': ['question^4', 'text'],
                        'type': 'best_fields'
                    }
                },
                'filter': [
                    {'term': {'course': 'machine-learning-zoomcamp'}}
                ]
            }
        },
        'size': 3  
    }
)

In [43]:
result_docs = []
for hit in response['hits']['hits']:
    result_docs.append(hit['_source'])

In [45]:
result_docs

[{'text': 'Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)',
  'section': '5. Deploying Machine Learning Models',
  'question': 'How do I debug a docker container?',
  'course': 'machine-learning-zoomcamp'},
 {'text': "You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:\nTo copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:\ndocker cp /path/to/local/file_or_directory container_id:/path/in/container\nHrithik Kumar Advani",
  'section': '5. Deploying Machine Learning Models',
  'question': 'How do I copy files from my local machine to docker container?',
 

In [55]:
def build_prompt(query, search_results):
    prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
""".strip()

    context = ""

    for doc in search_results:
        context = context + f"Q: {doc['question']}\nA: {doc['text']}\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

In [56]:
prompt = build_prompt(query, result_docs)

In [57]:
len(prompt)

1462

In [59]:
encoding = tiktoken.encoding_for_model("gpt-4o")
token = encoding.encode(prompt)
len(token)

322

In [61]:
client = OpenAI(
    base_url='http://localhost:11434/v1/',
    api_key='ollama',
)

In [62]:
response = client.chat.completions.create(model='phi3',messages=[{'role':'user','content': prompt }])

In [72]:
response.choices[0].message.content

" To execute a command in a running docker container, you can use the `docker exec` command with the `-it` option for interactive mode and specifying the container-id. Here's how to do it:\n\n1. First, ensure that your container is running using the following command to get its ID:\n   ```\n   docker ps (find the container-id)\n   ```\n\n2. Then, execute a command in the specific container by replacing `<container-id>` with the actual id you obtained from the previous step:\n   ```\n   docker exec -it <container-id> bash\n   ```"